# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,May 18 2022 1:55PM,237565,22,615085,"NBTY Global, Inc.",Released
1,May 18 2022 1:31PM,237563,10,AERO517-01,Aerogen LTD.,Released
2,May 18 2022 1:31PM,237563,10,AERO517-02,Aerogen LTD.,Released
3,May 18 2022 1:31PM,237563,10,AERO517-03,Aerogen LTD.,Released
4,May 18 2022 1:31PM,237563,10,AERO517-04,Aerogen LTD.,Released
5,May 18 2022 1:12PM,237542,10,0085679121,ISDIN Corporation,Released
6,May 18 2022 1:12PM,237542,10,0085679122,ISDIN Corporation,Released
7,May 18 2022 1:12PM,237542,10,0085679129,ISDIN Corporation,Released
8,May 18 2022 1:12PM,237542,10,0085679127,ISDIN Corporation,Released
9,May 18 2022 1:12PM,237542,10,0085679157,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
48,237558,Released,1
49,237561,Executing,1
50,237562,Released,5
51,237563,Released,4
52,237565,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
237558,NaN,NaN,1.0
237561,NaN,1.0,NaN
237562,NaN,NaN,5.0
237563,NaN,NaN,4.0
237565,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
237427,0.0,1.0,0.0
237428,0.0,0.0,1.0
237430,0.0,1.0,0.0
237431,26.0,1.0,1.0
237432,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
237427,0,1,0
237428,0,0,1
237430,0,1,0
237431,26,1,1
237432,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,237427,0,1,0
1,237428,0,0,1
2,237430,0,1,0
3,237431,26,1,1
4,237432,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,237427,,1,
1,237428,,,1
2,237430,,1,
3,237431,26,1,1
4,237432,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,May 18 2022 1:55PM,237565,22,"NBTY Global, Inc."
1,May 18 2022 1:31PM,237563,10,Aerogen LTD.
5,May 18 2022 1:12PM,237542,10,ISDIN Corporation
41,May 18 2022 1:07PM,237562,10,Emerginnova
46,May 18 2022 1:06PM,237561,10,"SD Head USA, LLC"
47,May 18 2022 12:38PM,237557,20,"ACI Healthcare USA, Inc."
75,May 18 2022 12:36PM,237558,19,"Emersa Waterbox, LLC"
76,May 18 2022 12:30PM,237554,10,"Methapharm, Inc."
77,May 18 2022 12:29PM,237553,19,"AdvaGen Pharma, Ltd"
78,May 18 2022 12:24PM,237550,20,Reliable 1 Laboratories LLC


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,May 18 2022 1:55PM,237565,22,"NBTY Global, Inc.",,,1
1,May 18 2022 1:31PM,237563,10,Aerogen LTD.,,,4
2,May 18 2022 1:12PM,237542,10,ISDIN Corporation,,,36
3,May 18 2022 1:07PM,237562,10,Emerginnova,,,5
4,May 18 2022 1:06PM,237561,10,"SD Head USA, LLC",,1,
5,May 18 2022 12:38PM,237557,20,"ACI Healthcare USA, Inc.",,,28
6,May 18 2022 12:36PM,237558,19,"Emersa Waterbox, LLC",,,1
7,May 18 2022 12:30PM,237554,10,"Methapharm, Inc.",,,1
8,May 18 2022 12:29PM,237553,19,"AdvaGen Pharma, Ltd",,,1
9,May 18 2022 12:24PM,237550,20,Reliable 1 Laboratories LLC,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 18 2022 1:55PM,237565,22,"NBTY Global, Inc.",1,,
1,May 18 2022 1:31PM,237563,10,Aerogen LTD.,4,,
2,May 18 2022 1:12PM,237542,10,ISDIN Corporation,36,,
3,May 18 2022 1:07PM,237562,10,Emerginnova,5,,
4,May 18 2022 1:06PM,237561,10,"SD Head USA, LLC",,1,
5,May 18 2022 12:38PM,237557,20,"ACI Healthcare USA, Inc.",28,,
6,May 18 2022 12:36PM,237558,19,"Emersa Waterbox, LLC",1,,
7,May 18 2022 12:30PM,237554,10,"Methapharm, Inc.",1,,
8,May 18 2022 12:29PM,237553,19,"AdvaGen Pharma, Ltd",1,,
9,May 18 2022 12:24PM,237550,20,Reliable 1 Laboratories LLC,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 18 2022 1:55PM,237565,22,"NBTY Global, Inc.",1,,
1,May 18 2022 1:31PM,237563,10,Aerogen LTD.,4,,
2,May 18 2022 1:12PM,237542,10,ISDIN Corporation,36,,
3,May 18 2022 1:07PM,237562,10,Emerginnova,5,,
4,May 18 2022 1:06PM,237561,10,"SD Head USA, LLC",,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 18 2022 1:55PM,237565,22,"NBTY Global, Inc.",1.0,NaN,NaN
1,May 18 2022 1:31PM,237563,10,Aerogen LTD.,4.0,NaN,NaN
2,May 18 2022 1:12PM,237542,10,ISDIN Corporation,36.0,NaN,NaN
3,May 18 2022 1:07PM,237562,10,Emerginnova,5.0,NaN,NaN
4,May 18 2022 1:06PM,237561,10,"SD Head USA, LLC",NaN,1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 18 2022 1:55PM,237565,22,"NBTY Global, Inc.",1.0,0.0,0.0
1,May 18 2022 1:31PM,237563,10,Aerogen LTD.,4.0,0.0,0.0
2,May 18 2022 1:12PM,237542,10,ISDIN Corporation,36.0,0.0,0.0
3,May 18 2022 1:07PM,237562,10,Emerginnova,5.0,0.0,0.0
4,May 18 2022 1:06PM,237561,10,"SD Head USA, LLC",0.0,1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3562856,124.0,25.0,27.0
12,237541,0.0,1.0,0.0
16,475079,16.0,7.0,0.0
18,1899876,7.0,16.0,0.0
19,1662449,5.0,23.0,8.0
20,1662429,34.0,4.0,1.0
22,237565,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3562856,124.0,25.0,27.0
1,12,237541,0.0,1.0,0.0
2,16,475079,16.0,7.0,0.0
3,18,1899876,7.0,16.0,0.0
4,19,1662449,5.0,23.0,8.0
5,20,1662429,34.0,4.0,1.0
6,22,237565,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,124.0,25.0,27.0
1,12,0.0,1.0,0.0
2,16,16.0,7.0,0.0
3,18,7.0,16.0,0.0
4,19,5.0,23.0,8.0
5,20,34.0,4.0,1.0
6,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,124.0
1,12,Released,0.0
2,16,Released,16.0
3,18,Released,7.0
4,19,Released,5.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,18,19,20,22
Status,,,,,,,
Completed,27.0,0.0,0.0,0.0,8.0,1.0,0.0
Executing,25.0,1.0,7.0,16.0,23.0,4.0,0.0
Released,124.0,0.0,16.0,7.0,5.0,34.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,18,19,20,22
0,Completed,27.0,0.0,0.0,0.0,8.0,1.0,0.0
1,Executing,25.0,1.0,7.0,16.0,23.0,4.0,0.0
2,Released,124.0,0.0,16.0,7.0,5.0,34.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,18,19,20,22
0,Completed,27.0,0.0,0.0,0.0,8.0,1.0,0.0
1,Executing,25.0,1.0,7.0,16.0,23.0,4.0,0.0
2,Released,124.0,0.0,16.0,7.0,5.0,34.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()